In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import matplotlib.pyplot as plt
import cv2
import numpy as np
from utils import *
from models import *
from data_utils import *


/home/sudeep/.conda/envs/keras_gpu_tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
VIDEOS_DIR, IMAGES_DIR, classes, class_to_index, videos = get_global_variables()

In [3]:
dcnn = dilated_conv((40, 172, 172, 3))

In [4]:
dcnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40, 172, 172, 0                                            
__________________________________________________________________________________________________
BatchNorm_1 (TimeDistributed)   (None, 40, 172, 172, 12          input_1[0][0]                    
__________________________________________________________________________________________________
Conv_1a (TimeDistributed)       (None, 40, 43, 43, 3 4736        BatchNorm_1[0][0]                
__________________________________________________________________________________________________
Conv_1b (TimeDistributed)       (None, 40, 43, 43, 3 9248        Conv_1a[0][0]                    
__________________________________________________________________________________________________
Pool_1 (Ti

In [7]:
dcnn.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.6],
        metrics=['accuracy'],
        optimizer='adam')

## DATASET

In [10]:
X_train.shape, X_test.shape,Y_train.shape, Y_test.shape

((340, 40, 172, 172, 3), (22,), (340,), (22,))

In [11]:
X_train, Y_train = permute(X_train, Y_train)
Y_train = convert_to_one_hot(Y_train,9)

In [12]:
Y_train2 = np.tile(Y_train, (40, 1, 1))
Y_train2 = Y_train2.transpose(1, 0, 2)

In [13]:
Y_test

array([0, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 6, 6, 7, 7, 7, 8, 8, 8])

## TRAINING

In [16]:
#filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_weights_only = False, save_best_only = False, mode='auto',period=1)
dcnn = load_model('temp_11.h5')
for i in range(1):
    dcnn.fit(X_train, [Y_train,Y_train2] , batch_size=64, epochs=2 , validation_split=0.1)
    dcnn.save('temp_'+ str(i+12) + '.h5')

Train on 306 samples, validate on 34 samples
Epoch 1/2
306/306 [==============================] - 149s 486ms/step - loss: 0.8307 - Output_loss: 0.1621 - time_distributed_3_loss: 1.1144 - Output_acc: 0.9412 - time_distributed_3_acc: 0.5799 - val_loss: 0.5596 - val_Output_loss: 0.1095 - val_time_distributed_3_loss: 0.7502 - val_Output_acc: 0.9706 - val_time_distributed_3_acc: 0.8309
Epoch 2/2
306/306 [==============================] - 162s 530ms/step - loss: 0.7857 - Output_loss: 0.1339 - time_distributed_3_loss: 1.0864 - Output_acc: 0.9641 - time_distributed_3_acc: 0.5955 - val_loss: 0.5360 - val_Output_loss: 0.1041 - val_time_distributed_3_loss: 0.7198 - val_Output_acc: 0.9706 - val_time_distributed_3_acc: 0.8404


## TESTING

In [19]:
dcnn = load_model('temp_12.h5')

In [20]:
evaluate(dcnn, X_test, Y_test, verbose = False)

Confusion Matrix
[[3 0 0 0 0 0 0 0 0]
 [0 0 2 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 1 1 0 0]
 [0 2 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 1 1]]


59.09090909090909

In [22]:
del X_train
del Y_train

NameError: name 'X_train' is not defined

In [22]:
evaluate(dcnn, X_test_full, Y_test_full, verbose = False)

Confusion Matrix
[[16  0  0  0  0  0  0  0  0]
 [ 0  8  2  0  0  0  0  0  0]
 [ 0  1  8  0  0  0  0  0  0]
 [ 0  0  1  9  0  0  0  0  0]
 [ 0  0  0  0 14  3  0  0  0]
 [ 0  0  0  0  0  5  0  0  0]
 [ 0  0  0  0  0  3  8  0  0]
 [ 0  2  0  0  0  0  0 17  0]
 [ 0  0  0  0  0  1  0  1 12]]


87.38738738738738

In [25]:
evaluate(dcnn, X_test_unseen, Y_test_unseen, verbose = False)

Confusion Matrix
[[3 0 0 0 0 1 0 0 0]
 [0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 0 0 1 2 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 4]]


68.18181818181817